In [ ]:
import polars as pl
import numpy as np
import networkx as nx
import random
import rtsvg
rt = rtsvg.RACETrack()
circles     = [(25,25,20),(80,80,30),(20,100,10),(25,60,5),(60,30,5),(48,48,6),(40,90,6)]
svg_hdr     = '<svg x="0" y="0" width="768" height="768" viewbox="0 0 128 128"><rect x="0" y="0" width="128" height="128" fill="white" />'
svg_ftr     = '</svg>'
svg_circles = []
for i in range(len(circles)):
    svg_circles.append(f'<circle cx="{circles[i][0]}" cy="{circles[i][1]}" r="{circles[i][2]}" fill="none" stroke="{rt.co_mgr.getColor(i)}" stroke-width="0.4"/>')
    svg_circles.append(f'<circle cx="{circles[i][0]}" cy="{circles[i][1]}" r="0.5" fill="{rt.co_mgr.getColor(i)}"/>')

#rt.tile([svg_hdr + ''.join(svg_circles) + svg_ftr])

In [2]:
# return bisector of points p0 and p1 -- bisector is ((x,y),(u,v)) where u,v is the vector of the bisector
def _isedgar_bisector(p0, p1):
    x, y     = (p0[0] + p1[0])/2.0, (p0[1] + p1[1])/2.0
    uv       = rt.unitVector((p0, p1))
    pdx, pdy = uv[1], -uv[0]
    return ((x,y), (pdx,pdy))
# For the circle version
def _isedgar_bisectorForCircles(c0, c1):
    uv              = rt.unitVector((c0, c1))
    x0, y0          = c0[0] + uv[0] * c0[2], c0[1] + uv[1] * c0[2]
    x1, y1          = c1[0] - uv[0] * c1[2], c1[1] - uv[1] * c1[2]
    x,  y           = (x0+x1)/2.0, (y0+y1)/2.0
    pdx, pdy        = uv[1], -uv[0]
    return ((x,y), (pdx,pdy))
# returns vertices that intersect the polygon
def _isedgar_intersects(bisects, poly, merge_threshold):
    inters, already_added = [], set()
    for i in range(0, len(poly)):
        p0, p1 = poly[i], poly[(i+1)%len(poly)]
        xy = rt.lineSegmentIntersectionPoint((bisects[0], (bisects[0][0] + bisects[1][0], bisects[0][1] + bisects[1][1])),(p0, p1))
        if xy is not None:
            xy = (round(xy[0], 2), round(xy[1], 2))
            too_close = False
            for pt in already_added:
                if rt.segmentLength((pt, xy)) < merge_threshold:
                    too_close = True
                    break
            if too_close == False:
                inters.append((xy, i, (i+1)%len(poly)))
                already_added.add(xy)
    return inters
# conctenate a point, a list, and a point into a new list
def _isedgar_createCell(my_cell, p0, p1, i0, i1, sign):
    l = [p0]
    i = i0
    while i != i1:
        l.append(my_cell[i])
        i += sign
        if i < 0:             i += len(my_cell)
        if i >= len(my_cell): i -= len(my_cell)
    l.append(my_cell[i1])
    l.append(p1)
    return l
# contain true if pt is in poly
def _isedgar_contains(poly, pt):
    inter_count = 0
    for i in range(len(poly)):
        p0, p1 = poly[i], poly[(i+1)%len(poly)]
        _tuple_ = rt.segmentsIntersect((pt,(pt[0]+1e9,pt[1])),(p0,p1)) # use a ray from the pt to test
        if _tuple_[0] and (_tuple_[1], _tuple_[2]) != p1: inter_count += 1
    if inter_count%2 == 1: return True
    return False
# remove any near duplicate points from the polygon
def _isedgar_removeDuplicatePoints(_poly_, merge_threshold):
    _new_ = []
    _new_.append(_poly_[0])
    for i in range(1,len(_poly_)):
        if rt.segmentLength((_poly_[i],_new_[-1])) > merge_threshold: _new_.append(_poly_[i])
    return _new_
# normalize the segment
def _isedgar_normalizeSegment(segment):
    _xy0_, _xy1_ = segment
    if _xy0_[0] <  _xy1_[0] or (_xy0_[0] == _xy1_[0] and _xy0_[1] < _xy1_[1]): return (_xy0_, _xy1_)
    else:                                                                      return (_xy1_, _xy0_)
# clumsy ... but this should work / if not very elegantly...
def _isedgar_accountForChanges(_old_, _new_, _p_, s2p):
    segs_in_old, segs_in_new, xy_to_seg_in_old, xy_to_seg_in_new = set(), set(), {}, {}
    for i in range(len(_old_)):
        _segment_ = _isedgar_normalizeSegment((_old_[i], _old_[(i+1)%len(_old_)]))
        segs_in_old.add(_segment_)
        if _segment_[0] not in xy_to_seg_in_old: xy_to_seg_in_old[_segment_[0]] = set()
        if _segment_[1] not in xy_to_seg_in_old: xy_to_seg_in_old[_segment_[1]] = set()
        xy_to_seg_in_old[_segment_[0]].add(_segment_)
        xy_to_seg_in_old[_segment_[1]].add(_segment_)
    for i in range(len(_new_)):
        _segment_ = _isedgar_normalizeSegment((_new_[i], _new_[(i+1)%len(_new_)]))
        segs_in_new.add(_segment_)
        if _segment_[0] not in xy_to_seg_in_new: xy_to_seg_in_new[_segment_[0]] = set()
        if _segment_[1] not in xy_to_seg_in_new: xy_to_seg_in_new[_segment_[1]] = set()
        xy_to_seg_in_new[_segment_[0]].add(_segment_)
        xy_to_seg_in_new[_segment_[1]].add(_segment_)
    segs_only_in_old = segs_in_old - segs_in_new
    segs_only_in_new = segs_in_new - segs_in_old
    # new should be three -- two should be shortened (and share a single xy with the old set), one should be new (both xy's unique)
    if len(segs_only_in_new) != 3: raise ValueError('segs_only_in_new != 3')
    seg0, seg1, seg2 = list(segs_only_in_new)
    if   seg0[0] not in xy_to_seg_in_old and seg0[1] not in xy_to_seg_in_old: # seg0 is new
        seg_new, seg_single0, seg_single1 = seg0, seg1, seg2
    elif seg1[0] not in xy_to_seg_in_old and seg1[1] not in xy_to_seg_in_old: # seg1 is new
        seg_new, seg_single0, seg_single1 = seg1, seg0, seg2
    elif seg2[0] not in xy_to_seg_in_old and seg2[1] not in xy_to_seg_in_old: # seg2 is new
        seg_new, seg_single0, seg_single1 = seg2, seg0, seg1
    else: raise ValueError('no match [new segment]')
    # transfer points to both seg_single0 and seg_single1
    # ... find the two possible transfers for seg_single0
    if   seg_single0[0] in xy_to_seg_in_old:
        if seg_single0[1] in xy_to_seg_in_old: raise ValueError('seg_single0[1] in xy_to_seg_in_old (and should not be)')
        single0_old_seg0, single0_old_seg1 = list(xy_to_seg_in_old[seg_single0[0]])
    elif seg_single0[1] in xy_to_seg_in_old:
        if seg_single0[0] in xy_to_seg_in_old: raise ValueError('seg_single0[0] in xy_to_seg_in_old (and should not be)')
        single0_old_seg0, single0_old_seg1 = list(xy_to_seg_in_old[seg_single0[1]])
    else: raise ValueError('no match [seg_single0]')

    # ... find the two possible transfers for seg_single1
    if   seg_single1[0] in xy_to_seg_in_old:
        if seg_single1[1] in xy_to_seg_in_old: raise ValueError('seg_single1[1] in xy_to_seg_in_old (and should not be)')
        single1_old_seg0, single1_old_seg1 = list(xy_to_seg_in_old[seg_single1[0]])
    elif seg_single1[1] in xy_to_seg_in_old:
        if seg_single1[0] in xy_to_seg_in_old: raise ValueError('seg_single1[0] in xy_to_seg_in_old (and should not be)')
        single1_old_seg0, single1_old_seg1 = list(xy_to_seg_in_old[seg_single1[1]])
    else: raise ValueError('no match [seg_single1]')

    if (single0_old_seg0 == single1_old_seg0 and single0_old_seg1 == single1_old_seg1) or \
        (single0_old_seg0 == single1_old_seg1 and single0_old_seg1 == single1_old_seg0):
        # special case where the two segments are the same -- happens when the new cell is a triangle
        s2p[seg_single0] = s2p[single0_old_seg0]
        s2p[seg_single1] = s2p[single0_old_seg1]
    else:
        if   single0_old_seg0 in segs_in_new: s2p[seg_single0] = s2p[single0_old_seg1]
        elif single0_old_seg1 in segs_in_new: s2p[seg_single0] = s2p[single0_old_seg0]
        else: raise ValueError('at least one single0_old should be in segs_in_new')
        if   single1_old_seg0 in segs_in_new: s2p[seg_single1] = s2p[single1_old_seg1]
        elif single1_old_seg1 in segs_in_new: s2p[seg_single1] = s2p[single1_old_seg0]
        else: raise ValueError('at least one single1_old should be in segs_in_new')
# create a merge lookup table for xy's that are too close
def _isedgar_createMergerLookup(cells, merge_threshold):
    xys = set()
    for cell in cells:
        for xy in cell: xys.add(xy)
    xy_becomes = {}
    for xy0 in xys:
        for xy1 in xys:
            if xy0 == xy1 or xy1 in xy_becomes: continue
            if rt.segmentLength((xy0, xy1)) < merge_threshold:
                xy_becomes[xy1] = xy0
    return xy_becomes


In [3]:
#
# Algorithm source & description from:
# "A simple algorithm for 2D Voronoi diagrams"
# https://www.youtube.com/watch?v=I6Fen2Ac-1U
# https://gist.github.com/isedgar/d445248c9ff6c61cef44fc275cb2398f
#
def isedgarVoronoi(self, S, Box=None, pad=10, use_circle_radius=False, merge_threshold=0.5):
    if Box is None:
        x_l, x_r, y_t, y_b = S[0][0], S[0][0], S[0][1], S[0][1]
        for pt in S: x_l, x_r, y_t, y_b = min(x_l, pt[0]), max(x_r, pt[0]), max(y_t, pt[1]), min(y_b, pt[1])
        Box = [(x_l-pad, y_t+pad), (x_r+pad, y_t+pad), (x_r+pad, y_b-pad), (x_l-pad, y_b-pad)]
    cells, all_segments_to_points = [], {}
    for i in range(len(S)):
        p                  = S[i]
        cell               = Box
        segments_to_points = {}   # do this on a local basis and then group w/ others at the end
        for j in range(len(cell)):
            segment = _isedgar_normalizeSegment((cell[j], cell[(j+1)%len(cell)]))
            if segment not in segments_to_points: segments_to_points[segment] = set()
            segments_to_points[segment].add(p[:2])
        for q in S:
            if p == q: continue
            B            = _isedgar_bisector(p,q) if use_circle_radius == False else _isedgar_bisectorForCircles(p,q)
            B_intersects = _isedgar_intersects(B, cell, merge_threshold)
            if len(B_intersects) == 2:
                t1, t2  = B_intersects[0][0], B_intersects[1][0] # these are _xy_ tuples
                segment = _isedgar_normalizeSegment((t1, t2))
                if segment not in segments_to_points: segments_to_points[segment] = set()
                segments_to_points[segment].add(p[:2]), segments_to_points[segment].add(q[:2]) 
                xi, xj  = B_intersects[0][2], B_intersects[1][1]
                newCell = _isedgar_createCell(cell, t1, t2, xi, xj, 1)
                if _isedgar_contains(newCell, p) == False:
                    xi, xj  = B_intersects[0][1], B_intersects[1][2]
                    newCell = _isedgar_createCell(cell, t1, t2, xi, xj, -1)
                _isedgar_accountForChanges(cell, newCell, p[:2], segments_to_points)
                cell = newCell
        cells.append(cell)
        # transfer segments from local to all
        for j in range(len(cell)):
            segment = _isedgar_normalizeSegment((cell[j], cell[(j+1)%len(cell)]))
            if segment not in all_segments_to_points: all_segments_to_points[segment] = set()
            all_segments_to_points[segment].update(segments_to_points[segment])
        # remove any near duplicate points
        xy_becomes = _isedgar_createMergerLookup(cells, merge_threshold)
        new_cells = []
        for cell in cells:
            new_cell = []
            for i in range(len(cell)):
                if cell[i] in xy_becomes: new_cell.append(xy_becomes[cell[i]])
                else:                     new_cell.append(cell[i])
            new_cells.append(new_cell)
        cells = new_cells
        new_all_segments_to_points = {}
        for _segment_ in all_segments_to_points:
            if _segment_[0] in xy_becomes: _xy0_ = xy_becomes[_segment_[0]]
            else:                          _xy0_ = _segment_[0]
            if _segment_[1] in xy_becomes: _xy1_ = xy_becomes[_segment_[1]]
            else:                          _xy1_ = _segment_[1]
            new_all_segments_to_points[(_xy0_, _xy1_)] = all_segments_to_points[_segment_]
        all_segments_to_points = new_all_segments_to_points

    return cells, all_segments_to_points

In [ ]:
polys, s2p = isedgarVoronoi(rt, circles, [(0,0),(0,128),(128,128),(128,0)], merge_threshold=0.5, use_circle_radius=True)
svg_polys, xys_seen = [], set()
for i in range(len(polys)):
    poly = polys[i]
    d = [f'M {poly[0][0]} {poly[0][1]} ']
    xys_seen.add(poly[0])
    for j in range(1,len(poly)): 
        d.append(f'L {poly[j][0]} {poly[j][1]} ')
        xys_seen.add(poly[j])
    d.append('Z')
    svg_polys.append(f'<path d="{"".join(d)}" fill="none" stroke="{rt.co_mgr.getColor(i)}" stroke-width="0.5"/>')
for _xy_ in xys_seen:
    svg_polys.append(f'<circle cx="{_xy_[0]}" cy="{_xy_[1]}" r="{1.0 + random.random()}" fill="none" stroke="#000000" stroke-width="0.1"/>')
svg_s2p = []
for _segment_ in s2p:
    x0,y0,x1,y1 = _segment_[0][0], _segment_[0][1], _segment_[1][0], _segment_[1][1]
    for _xy_ in s2p[_segment_]:
        color_i = 0
        for i in range(len(circles)):
            if _xy_ == circles[i][:2]:
                color_i = i
                break
        svg_s2p.append(f'<line x1="{x0}" y1="{y0}" x2="{_xy_[0]}" y2="{_xy_[1]}" stroke="{rt.co_mgr.getColor(color_i)}" stroke-width="0.5"/>')
        svg_s2p.append(f'<line x1="{_xy_[0]}" y1="{_xy_[1]}" x2="{x1}" y2="{y1}" stroke="{rt.co_mgr.getColor(color_i)}" stroke-width="0.5"/>')
for _segment_ in s2p:
    x0,y0,x1,y1 = _segment_[0][0], _segment_[0][1], _segment_[1][0], _segment_[1][1]
    svg_s2p.append(f'<line x1="{x0}" y1="{y0}" x2="{x1}" y2="{y1}" stroke="#000000" stroke-width="0.05"/>')
print(f'{len(xys_seen)=}') # 30 points seen (base version w/out any mergers)
                           # 27 if intersection points are rounded to 1 decimal place (27 also for 2 decimal place rounding)
rt.tile([svg_hdr + ''.join(svg_circles) + ''.join(svg_polys) + svg_ftr,
         svg_hdr + ''.join(svg_circles) + ''.join(svg_s2p)   + svg_ftr])

In [ ]:
min_segment_length = 1e9
for i in range(len(polys)):
    _poly_ = polys[i]
    for j in range(len(_poly_)):
        _xy0_, _xy1_ = _poly_[j], _poly_[(j+1)%len(_poly_)]
        min_segment_length = min(min_segment_length, rt.segmentLength((_xy0_, _xy1_)))
print(min_segment_length) # 7.53... for no merger version

In [6]:
tiles = []
for i in range(len(circles)):
    _xy_ = circles[i][:2]
    single_circle_svg = []
    for _segment_ in s2p:
        if _xy_ in s2p[_segment_]:
            x0,y0,x1,y1 = _segment_[0][0], _segment_[0][1], _segment_[1][0], _segment_[1][1]
            single_circle_svg.append(f'<line x1="{x0}" y1="{y0}" x2="{x1}" y2="{y1}" stroke="{rt.co_mgr.getColor(i)}" stroke-width="0.5"/>')
    _poly_ = polys[i]
    d = [f'M {_poly_[0][0]} {_poly_[0][1]} ']
    for j in range(1,len(_poly_)): d.append(f'L {_poly_[j][0]} {_poly_[j][1]} ')
    d.append('Z')
    single_circle_svg.append(f'<path d="{"".join(d)}" fill="none" stroke="#000000" stroke-width="0.1"/>')
    tiles.append(svg_hdr + ''.join(svg_circles) + ''.join(single_circle_svg) + svg_ftr)
#rt.table(tiles, per_row=2, spacer=10)

In [ ]:
svg_leftovers, svg_no_edge_found, xys_seen, pieces = [], [], set(), set()
for i in range(len(circles)):
    _xy_, _poly_ = circles[i][:2], polys[i]
    for _segment_ in s2p:
        if _xy_ in s2p[_segment_]:
            for j in range(len(_poly_)):
                _edge_ = (_poly_[j], _poly_[(j+1)%len(_poly_)])
                if rt.segmentsOverlap(_segment_, _edge_): break
                else:                                     _edge_ = None
            if _edge_ is None: 
                svg_no_edge_found.append(f'<line x1="{_segment_[0][0]}" y1="{_segment_[0][1]}" x2="{_segment_[1][0]}" y2="{_segment_[1][1]}" stroke="{rt.co_mgr.getColor(i)}" stroke-width="1.0"/>')
                _segment_ = ((round(_segment_[0][0], 2), round(_segment_[0][1], 2)), (round(_segment_[1][0], 2), round(_segment_[1][1], 2)))
                pieces.add(_isedgar_normalizeSegment(_segment_))
            else:
                diffs = rt.segmentDiffPieces(_segment_, _edge_)
                for _piece_ in diffs:
                    _piece_ = ((round(_piece_[0][0], 2), round(_piece_[0][1], 2)), (round(_piece_[1][0], 2), round(_piece_[1][1], 2)))
                    xys_seen.add(_piece_[0]), xys_seen.add(_piece_[1]), pieces.add(_isedgar_normalizeSegment(_piece_))
                    svg_leftovers.append(f'<line x1="{_piece_[0][0]}" y1="{_piece_[0][1]}" x2="{_piece_[1][0]}" y2="{_piece_[1][1]}" stroke="{rt.co_mgr.getColor(i)}" stroke-width="1.0"/>')
                    svg_leftovers.append(f'<line x1="{_piece_[0][0]}" y1="{_piece_[0][1]}" x2="{_piece_[1][0]}" y2="{_piece_[1][1]}" stroke="#000000" stroke-width="0.1"/>')
for _xy_ in xys_seen:
    svg_leftovers.append(f'<circle cx="{_xy_[0]}" cy="{_xy_[1]}" r="{1.0 + random.random()}" fill="none" stroke="#000000" stroke-width="0.1"/>')

rt.tile([svg_hdr + ''.join(svg_circles) + ''.join(svg_leftovers) + ''.join(svg_no_edge_found) + svg_ftr])

In [8]:
pieces_ls = list(pieces)
connected = {}
_factor_  = 4.0
for _piece_i_ in range(len(pieces_ls)):
    _piece_     = pieces_ls[_piece_i_]
    _piece_len_ = rt.segmentLength(_piece_)
    for _other_i_ in range(_piece_i_+1, len(pieces_ls)):
        _other_     = pieces_ls[_other_i_]
        _other_len_ = rt.segmentLength(_other_)
        d0, d1, d2, d3 = rt.segmentLength((_piece_[0], _other_[0])), rt.segmentLength((_piece_[0], _other_[1])), rt.segmentLength((_piece_[1], _other_[0])), rt.segmentLength((_piece_[1], _other_[1]))
        if (d0 < _piece_len_/_factor_ and d0 < _other_len_/_factor_) or \
           (d1 < _piece_len_/_factor_ and d1 < _other_len_/_factor_) or \
           (d2 < _piece_len_/_factor_ and d2 < _other_len_/_factor_) or \
           (d3 < _piece_len_/_factor_ and d3 < _other_len_/_factor_):
            if _piece_ not in connected: connected[_piece_] = set()
            connected[_piece_].add(_other_), connected[_piece_].add(_piece_)
            if _other_ not in connected: connected[_other_] = set()
            connected[_other_].add(_piece_), connected[_other_].add(_other_)

In [ ]:
def isTriangle(_segments_, merge_threshold=0.5):
    if len(_segments_) < 3: return False
    xys = set()
    for _segment_ in _segments_: xys.add(_segment_[0]), xys.add(_segment_[1])
    xys_ls = list(xys)
    _centers_, _members_ = rt.kMeans2D(xys_ls, 3)
    # Are the clusters compact enough?
    for cluster_i in _centers_:
        _center_ = _centers_[cluster_i]
        for xy in _members_[cluster_i]:
            if rt.segmentLength((_center_, xy)) > merge_threshold: 
                return False
    # Do the edges form a triangle?
    # ... make a map from the points seen to their cluster numbers
    xy_to_cluster = {}
    for cluster_i in _members_: 
        for xy in _members_[cluster_i]: 
            xy_to_cluster[xy] = cluster_i
    # ... convert the edges to their cluster numbers
    _edges_seen_ = set()
    for _segment_ in _segments_:
        cluster_0 = xy_to_cluster[_segment_[0]]
        cluster_1 = xy_to_cluster[_segment_[1]]
        if cluster_0 == cluster_1: return False
        if cluster_0 > cluster_1: cluster_0, cluster_1 = cluster_1, cluster_0
        _edges_seen_.add((cluster_0, cluster_1))
    # Must be three edges exactly and they must be the edges of a triangle
    if len(_edges_seen_) != 3: return False
    if (0,1) not in _edges_seen_ or \
       (1,2) not in _edges_seen_ or \
       (0,2) not in _edges_seen_: return False
    return True

_svg_                    = []
_component_no_           = 1
component_no_to_segments = {}
already_rendered         = set()
for _segment_ in connected:
    if _segment_ in already_rendered: continue
    _color_ = rt.co_mgr.getColor(_segment_)
    _tile_  = []
    for _other_ in connected[_segment_]:
        x0,y0,x1,y1 = _other_[0][0], _other_[0][1], _other_[1][0], _other_[1][1]
        _svg_.append(f'<line x1="{x0}" y1="{y0}" x2="{x1}" y2="{y1}" stroke="{_color_}" stroke-width="0.4"/>')
        already_rendered.add(_other_)
    for _other_ in connected[_segment_]:
        x0,y0,x1,y1 = _other_[0][0], _other_[0][1], _other_[1][0], _other_[1][1]
        _svg_.append(f'<line x1="{x0}" y1="{y0}" x2="{x1}" y2="{y1}" stroke="#000000" stroke-width="0.05"/>')
    _color_txt_ = '#000000' if isTriangle(connected[_segment_]) else '#ff0000'   
    _svg_.append(rt.svgText(f'{_component_no_}', _segment_[0][0], _segment_[0][1], txt_h=5, color=_color_txt_))
    component_no_to_segments[_component_no_] = connected[_segment_]
    print(f'{_component_no_=} {len(connected[_segment_])=} {isTriangle(connected[_segment_])}')
    #print('\t', connected[_segment_])
    _component_no_ += 1

rt.tile([svg_hdr + ''.join(_svg_) + svg_ftr])

In [ ]:
component_no_to_segments[4]

In [ ]:
component_no_to_segments[5]